# Interfere Jina Embedding Model Package from AWS Marketplace

This notebook shows you how to deploy [jina-embedding-model](link) using Amazon SageMaker.

## Pre-requisites:
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**
1. To deploy this ML model successfully, ensure that:
    1. Either your IAM role has these three permissions and you have authority to make AWS Marketplace subscriptions in the AWS account used: 
        1. **aws-marketplace:ViewSubscriptions**
        1. **aws-marketplace:Unsubscribe**
        1. **aws-marketplace:Subscribe**  
    2. or your AWS account has a subscription to [jina-embedding-model](link). If so, skip step: [Subscribe to the model package](#1.-Subscribe-to-the-model-package)

## Contents:
1. [Subscribe to the model package](#1.-Subscribe-to-the-model-package)
2. [Real-time inference](#2.-Real-time-inference)
   1. [Create an endpoint with static instances](#A.-Create-an-endpoint-with-static-instances)
   2. [Create an endpoint that automatically scales](#B.-Create-an-endpoint-that-automatically-scales)
   3. [Create an serverless endpoint](#C.-Create-an-serverless-endpoint)
   4. [Perform real-time inference](#D.-Perform-real-time-inference)
3. [Batch inference](#3.-Batch-inference)
4. [Clean-up](#4.-Clean-up)
    1. [Delete the model](#A.-Delete-the-model)
    2. [Unsubscribe to the listing (optional)](#B.-Unsubscribe-to-the-listing-(optional))

# 1. Subscribe to the model package

In [2]:
# !pip install --upgrade jina-sagemaker

In [1]:
from jina_sagemaker import Client
import boto3

region = boto3.Session().region_name

# Specify the role if needed
role = ""
role = 'arn:aws:iam::253352124568:role/service-role/AmazonSageMaker-ExecutionRole-20230527T104084'

# Specify the model name
model_name = "jina-embedding-l-en-v1"

# Mapping for Model Packages
model_package_map = {
    "us-east-1": f"arn:aws:sagemaker:us-east-1:253352124568:model-package/{model_name}",
}

# Specify the model you want to use
if region not in model_package_map.keys():
    raise Exception(f"Current boto3 session region {region} is not supported.")

model_package_arn = model_package_map[region]

# 2. Real-time inference

To learn about real-time inference capabilities in Amazon SageMaker, please refer to [Documentations](https://docs.aws.amazon.com/sagemaker/latest/dg/realtime-endpoints.html).

## A. Create an endpoint with static instances

In [ ]:
client = Client(region_name=region)
endpoint_name = "my-endpoint"

In [2]:
# Create a new endpoint
client.create_endpoint(
    arn=model_package_arn, 
    role=role, 
    endpoint_name=endpoint_name, 
    instance_type="ml.g4dn.xlarge", 
    n_instances=1,
)

# OR, connect to it -if the endpoint is already created.
client.connect_to_endpoint(endpoint_name="my-endpoint")

## B. Perform real-time inference

In [8]:
result = client.embed(texts=[
    "how is the weather today", 
    "what is the weather like today",
    "what's the color of an orange",
])
result

[{'id': 'a95fe0b704d5cab6013777531a78229e',
  'embedding': [-1.9868216e-08,
   -0.019838968,
   -0.030698141,
   -0.036247253,
   0.06848526,
   0.0016110739,
   0.024970055,
   -0.08946291,
   0.017534891,
   -0.06556702,
   0.0,
   -0.08461952,
   -0.009290556,
   -0.024345398,
   0.10044607,
   0.02361711,
   -0.165905,
   0.124061584,
   0.05020841,
   0.0,
   0.16478793,
   -5.265077e-07,
   -0.15244198,
   -0.12791061,
   -0.036828995,
   0.07945887,
   0.06786855,
   0.009222667,
   0.03775279,
   -0.04062144,
   -0.13116455,
   -0.05202675,
   -0.09332085,
   -0.022381863,
   -0.07049179,
   0.052879333,
   0.014085134,
   0.0283432,
   -0.027805964,
   -0.1397543,
   -0.06768417,
   -0.059841752,
   0.0,
   -0.012865384,
   0.030080795,
   0.0153415995,
   0.06336403,
   -0.01011769,
   0.11812083,
   -0.027895292,
   0.007121404,
   -0.05778758,
   0.020668665,
   0.036275227,
   0.0181427,
   0.048617046,
   0.010211945,
   -0.014918963,
   0.0,
   0.0,
   -0.091901146,
   -

In [11]:
from scipy.spatial.distance import cosine

print(f'Cosine distance between the first 2 embeddings: {cosine(result[0]["embedding"], result[1]["embedding"])}')

print(f'Cosine distance between the first and the third embeddings: {cosine(result[0]["embedding"], result[2]["embedding"])}')
print(f'Cosine distance between the second and the third embeddings: {cosine(result[1]["embedding"], result[2]["embedding"])}')

Cosine distance between the first 2 embeddings: 0.02327896772684701
Cosine distance between the first and the third embeddings: 0.9639122464666876
Cosine distance between the second and the third embeddings: 0.9532006759243141


# 3. Clean-up

## A. Delete the model

In [ ]:
client.delete_endpoint()
client.close()

## B. Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.
